# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## 1 Handling the data

### 1.1 Converting F.A.Q from .txt to .csv file

In [3]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [4]:
questions, answers = get_questions_and_answers(faq_path="../data/faqs/faq_example.txt")
create_csv(questions=questions, answers=answers, destination_path="../data/faqs/", file_name="faq_example")

## 2 Testing LLMs techniques

### 2.1 Sentence Embeddings pretrained models

In [5]:
from sentence_transformers import SentenceTransformer, util
import spacy
import re


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = model.encode(dataframe.Question)
    message_embed = model.encode(get_clean_text(message))

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index


def get_clean_text(text:str):
    nlp_model = spacy.load("pt_core_news_lg")

    text_tokens = nlp_model(text)

    clean_text = []

    for token in text_tokens:
        if not token.is_stop and not token.is_punct:
            clean_text.append(token.text)

    clean_text = " ".join(clean_text)

    return re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', clean_text).lower()

/home/tanabe/faq-recommendations/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df = pd.read_csv("../data/faqs/faq_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [8]:
message = "Como que eu faço para contratar um serviço melhor?"
message = get_clean_text(message)
print(message)

contratar serviço melhor


#### 2.1.1 all-mpnet-base-v2 model

In [9]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 684kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 405kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 24.3MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 471kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 22.4MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:31<00:00, 13.9MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 173kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 265kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.94MB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 1.60MB/s]
train_script.py: 100%|██████████| 13.1k/13.1k [00:00<00:00, 37.2MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 921kB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 1.03MB/s]


contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."


#### 2.1.2 all-MiniLM-L6-v2 model

In [10]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 4.45MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.03MB/s]
README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 21.0MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 367kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 28.3MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:06<00:00, 13.9MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 114kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 435kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.97MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 1.80MB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 49.8MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.04MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 160kB/s]


contratar serviço melhor


,Question,Answer
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."


#### 2.1.3 msmarco-distilbert-base-v4

In [11]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 538kB/s]
README.md: 100%|██████████| 3.75k/3.75k [00:00<00:00, 20.8MB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 464kB/s]
pytorch_model.bin: 100%|██████████| 265M/265M [00:19<00:00, 13.3MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 189kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 492kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.08MB/s]
tokenizer_config.json: 100%|██████████| 319/319 [00:00<00:00, 1.48MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 18.6MB/s]
modules.json: 100%|██████████| 229/229 [00:00<00:00, 319kB/s]


contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
15,"É possível contratar serviços adicionais, como...","Sim, oferecemos opções de streaming em parceri..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...


#### 2.1.4 bert-large-portuguese-cased (Finetuned model)

In [18]:
model = SentenceTransformer("../models/bert-large-portuguese-cased")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

No sentence-transformers model found with name ../models/bert-large-portuguese-cased. Creating a new one with MEAN pooling.


contratar serviço melhor


,Question,Answer
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
21,A empresa disponibiliza serviços de telefonia ...,"Sim, oferecemos opções para ligações internaci..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
8,osso fazer a contratação de serviços adicionai...,"Sim, você pode personalizar seu pacote de TV p..."
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
29,A empresa disponibiliza pacotes de internet co...,"Sim, temos opções de planos com tráfego ilimit..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...


## 3 Preparing to Deploy

In [19]:
from sentence_transformers import models

word_embedding_model = models.Transformer("../models/bert-large-portuguese-cased")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

embedding_model = SentenceTransformer(modules=[model, pooling_model])

In [20]:
nlp_model = spacy.load("pt_core_news_lg")

In [21]:
df = pd.read_csv("../data/faqs/faq_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


### 3.1 Preparing Data

#### 3.1.1 Creating Embeddings for each question and exporting as CSV file

In [23]:
def get_sentence_embeddings(model, series):
    """This function removes all 'stop words' for each sentence in a pandas.Series
    and then creates a embed for the sentence without the stop words."""

    for i, text in enumerate(series):
        embed = model.encode(get_clean_text(text))
        
        if i == 0:
            text_embeddings = pd.DataFrame({f"{i}": embed})
        else:
            text_embeddings[f"{i}"] = embed

    return text_embeddings

In [24]:
df_embedding = get_sentence_embeddings(embedding_model, df.Question)
df_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.188625,0.400795,0.411760,0.109736,0.009841,0.034626,0.166422,0.056301,0.097396,-0.092918,...,-0.413871,-0.321137,0.878566,-0.406816,-0.006660,0.082112,-0.196087,-0.239689,0.049381,-0.480112
1,0.669775,-0.513456,0.729793,0.126030,0.267571,0.481286,1.141500,0.142641,0.742800,0.549268,...,0.589085,0.127874,0.220500,0.422829,-0.126928,0.728831,0.353417,0.152385,-0.145057,0.394540
2,-0.381558,-0.054400,-0.294071,0.117846,-0.608610,-0.065049,-0.305379,0.236780,-0.003630,0.204660,...,0.205727,-0.050293,0.137386,-0.305943,-0.038120,-0.013327,-0.102131,-0.320342,0.274796,-0.026569
3,0.454017,0.232535,0.188409,-0.255085,0.524477,0.415265,-0.067255,0.152117,0.100764,0.834602,...,-0.138930,0.220053,0.151821,0.536146,0.055318,0.060070,0.320547,0.123528,-0.049228,0.277206
4,0.066648,-0.130414,0.453749,-0.097490,-0.049005,-0.478444,0.460946,0.129103,0.424206,-0.115655,...,0.023551,0.136368,0.457364,0.372164,0.002984,0.833865,0.270271,-0.148330,0.182577,0.618017


In [25]:
df_embedding.to_csv(path_or_buf="../data/embeddings/embeddings.csv", sep=";", index=False)

### 3.2 Exporting the model with the best performance

#### 3.2.1 Embeddings model

In [26]:
embedding_model.save('../models/embeddings_model_pt')

#### 3.2.2 NLP Model

In [27]:
nlp_model.to_disk('../models/nlp_model')